In [10]:
import os
import cv2
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt


def parse_cell_annotations(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_filename = root.find("filename").text
    cells = []

    for obj in root.findall(".//tablecell"):
        x0 = int(obj.find("x0").text)
        x1 = int(obj.find("x1").text)
        y0 = int(obj.find("y0").text)
        y1 = int(obj.find("y1").text)
        cells.append(((x0, y0, x1, y1)))

    return image_filename, cells


def xml2yolo(xml_path, image_dir, yolo_dir):
    image_filename, cells = parse_cell_annotations(xml_path)

    image_path = os.path.join(image_dir, image_filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image {image_filename} not found.")
        return

    image_height, image_width = image.shape[:2]
    yolo_filename = os.path.splitext(image_filename)[0] + ".txt"
    yolo_path = os.path.join(yolo_dir, yolo_filename)

    with open(yolo_path, "w") as f:
        for x0, y0, x1, y1 in cells:
            x_center = (x0 + x1) / 2 / image_width
            y_center = (y0 + y1) / 2 / image_height
            width = (x1 - x0) / image_width
            height = (y1 - y0) / image_height
            f.write(f"2 {x_center} {y_center} {width} {height}\n")



def visualize_cells(cell_xml_dir, image_dir):
    xml_files = [f for f in os.listdir(cell_xml_dir) if f.endswith(".xml")]

    for idx, xml_file in enumerate(xml_files):
        xml_path = os.path.join(cell_xml_dir, xml_file)
        image_filename, cells = parse_cell_annotations(xml_path)

        image_path = os.path.join(image_dir, image_filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image {image_filename} not found.")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        for x0, y0, x1, y1 in cells:
            cv2.rectangle(image, (x0, y0), (x1, y1), (255, 0, 0), 2)
            cv2.putText(image, "cell", ((x1-x0)//2 + x0, (y1-y0)//2 + y0), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0,0), 2)

        plt.figure(figsize=(10, 6))
        plt.imshow(image)
        plt.title(image_filename)
        plt.axis("off")
        plt.show()
        if idx == 10:
            break


In [11]:

ROOT = r"X:\tsr\TUCD"
CELL_XML_DIR = os.path.join(ROOT, "cell_xml")
IMAGE_DIR = os.path.join(ROOT, "images")
# visualize_cells(CELL_XML_DIR, IMAGE_DIR)

YOLO_DIR = os.path.join(ROOT, "labels")
os.makedirs(YOLO_DIR, exist_ok=True)

for xml_file in os.listdir(CELL_XML_DIR):
    xml_path = os.path.join(CELL_XML_DIR, xml_file)
    xml2yolo(xml_path, IMAGE_DIR, YOLO_DIR)

print("Done.")




Done.


In [4]:
import os
import random 
def find_images(p):
    images = []
    for root, _, files in os.walk(p):
        for f in files:
            if f.endswith(("jpg", "jpeg", "png")):
                images.append(os.path.join(root, f))

    return images

def build_yolo_listfiles(train_paths:list, val_paths:list, outdir:str):
    out_trainfile = os.path.join(outdir, "train.txt")
    out_valfile = os.path.join(outdir, "val.txt")

    with open(out_trainfile, "w") as f:
        for pth, max_n in train_paths.items():
            images = find_images(pth)
            if len(images) > max_n:
                images = random.sample(images, max_n)
            for img in images:
                f.write(img + "\n")

    with open(out_valfile, "w") as f:
        for pth, max_n in val_paths.items():
            images = find_images(pth)
            if len(images) > max_n:
                images = random.sample(images, max_n)
            for img in images:
                f.write(img + "\n")

TRAINPATHS = {r"X:\tsr\TUCD\images":1000, r"X:\tsr\TD-TSR\images\train":1000}
VALPATHS = {r"X:\tsr\TD-TSR\images\val":50}
OUTDIR = r"X:\tsr"

build_yolo_listfiles(TRAINPATHS, VALPATHS, OUTDIR)